In [1]:
%cd ..
#import site
#site.addsitedir('C:/Users/Admin/Documents/GitHub/blokus/')  # Replace with the actual path to your project folder
from src import bot
import src.objects as o
from src.objects import Player, Piece
from pygame import display, Surface, font, image, surfarray, Rect
import pygame
import src.events as e
import src.views as v
from os.path import join
import src.controllers as c
import numpy as np
import src.reward as r

c:\Users\Admin\Documents\GitHub\blokus
pygame 2.5.2 (SDL 2.28.3, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import numpy as np

In [3]:
from src import bot
board = o.LinkedGrid(20,20, 20)
players = []
colors = ["r", "g", "b", "y"]
for p in range(4):
    players.append(Player(colors[p]))
p_i = 0

In [4]:
player = players[0]
import json
path = 'data/all_playable_moves.json'

In [5]:
all_playable_moves = bot.get_all_playable_moves('data/all_playable_moves.json')


In [9]:
import copy

In [20]:
all_playable_moves = bot.get_all_playable_moves('data/all_playable_moves.json')
a = {key: {**all_playable_moves[key]} for key in all_playable_moves.keys()}

In [21]:
a[(0,0)][0] = 2
all_playable_moves[(0,0)][0] = 3
print(a[(0,0)][0], all_playable_moves[(0,0)][0])

2 3


2

In [31]:
playable_board, playable_pos = bot.get_playable_conditions((bot.convert_matrix_to_nparray(board.matrix)),bot.convert_color_to_number(player.c))
playable_pos = [(i,j) for i in range(20) for j in range(20)]
all_pos_play = {(i,j):{} for i in range(20) for j in range(20)}

piece_num = 0
for pkey in player.pieces.keys():
    for pindex in range(len(player.pieces[pkey])):
        piece = player.pieces[pkey][pindex].m
        piece_permutations = [piece, np.rot90(piece,1), np.rot90(piece,2), np.rot90(piece,3), np.fliplr(piece), np.rot90(np.fliplr(piece),1), np.rot90(np.fliplr(piece),2), np.rot90(np.fliplr(piece),3)]  
        for p in piece_permutations:
            for position in playable_pos:
                for i in range(len(p)):
                    for j in range(len(p[0])):
                        relative_coords = bot.get_piece_relative_coordinates(p, i,j)
                        absolute_coords = bot.get_absolute_coordinates(position, relative_coords)
                        if position in absolute_coords:
                            if piece_num not in all_pos_play[position].keys():
                                all_pos_play[position][piece_num] = []
                                
                            if bot.check_absolute_coordinates(playable_board, absolute_coords):
                                if [(pkey,pindex),absolute_coords] not in all_pos_play[position][piece_num]:
                                    all_pos_play[position][piece_num].append([(pkey,pindex),absolute_coords])
        piece_num+=1
string_keys_dict = {str(key): value for key, value in all_pos_play.items()}
    # Open the file in write mode and use json.dump() to save the list as a JSON file
with open(path, 'w') as json_file:
    json.dump(string_keys_dict, json_file)

In [26]:
with open(path, 'r') as json_file:
    string_keys_dict = json.load(json_file)
    # Convert string keys back to tuples
loaded_dict = {tuple(eval(key)): value for key, value in string_keys_dict.items()}

for pos in loaded_dict.keys():
    loaded_dict[pos] = {int(eval(key)): value for key, value in loaded_dict[pos].items()}

In [29]:
loaded_dict[(0,0)][0]

[[[1, 0], [[0, 0]]]]

In [17]:
string_keys_dict['(0, 0)'][0]

[[(1, 0), [(0, 0)]]]

In [13]:
piece_dict = {}
i = 0
for key in player.pieces.keys():
    for index,p in enumerate(player.pieces[key]):
        piece_dict[i] = p.m
        i+=1
        

        

In [14]:
piece_dict

{0: array([[1]]),
 1: array([[1],
        [1]]),
 2: array([[1],
        [1],
        [1]]),
 3: array([[1, 0],
        [1, 1]]),
 4: array([[1],
        [1],
        [1],
        [1]]),
 5: array([[0, 1],
        [0, 1],
        [1, 1]]),
 6: array([[1, 0],
        [1, 1],
        [1, 0]]),
 7: array([[1, 1],
        [1, 1]]),
 8: array([[1, 1, 0],
        [0, 1, 1]]),
 9: array([[1],
        [1],
        [1],
        [1],
        [1]]),
 10: array([[0, 1],
        [0, 1],
        [0, 1],
        [1, 1]]),
 11: array([[0, 1],
        [0, 1],
        [1, 1],
        [1, 0]]),
 12: array([[0, 1],
        [1, 1],
        [1, 1]]),
 13: array([[1, 1],
        [0, 1],
        [1, 1]]),
 14: array([[1, 0],
        [1, 1],
        [1, 0],
        [1, 0]]),
 15: array([[0, 1, 0],
        [0, 1, 0],
        [1, 1, 1]]),
 16: array([[1, 0, 0],
        [1, 0, 0],
        [1, 1, 1]]),
 17: array([[1, 1, 0],
        [0, 1, 1],
        [0, 0, 1]]),
 18: array([[1, 0, 0],
        [1, 1, 1],
        

In [20]:
player.pieces[5][11].m

array([[0, 1, 0],
       [1, 1, 1],
       [0, 1, 0]])

In [27]:
string_keys_dict.keys()

dict_keys(['(0, 0)', '(0, 1)', '(0, 2)', '(0, 3)', '(0, 4)', '(0, 5)', '(0, 6)', '(0, 7)', '(0, 8)', '(0, 9)', '(0, 10)', '(0, 11)', '(0, 12)', '(0, 13)', '(0, 14)', '(0, 15)', '(0, 16)', '(0, 17)', '(0, 18)', '(0, 19)', '(1, 0)', '(1, 1)', '(1, 2)', '(1, 3)', '(1, 4)', '(1, 5)', '(1, 6)', '(1, 7)', '(1, 8)', '(1, 9)', '(1, 10)', '(1, 11)', '(1, 12)', '(1, 13)', '(1, 14)', '(1, 15)', '(1, 16)', '(1, 17)', '(1, 18)', '(1, 19)', '(2, 0)', '(2, 1)', '(2, 2)', '(2, 3)', '(2, 4)', '(2, 5)', '(2, 6)', '(2, 7)', '(2, 8)', '(2, 9)', '(2, 10)', '(2, 11)', '(2, 12)', '(2, 13)', '(2, 14)', '(2, 15)', '(2, 16)', '(2, 17)', '(2, 18)', '(2, 19)', '(3, 0)', '(3, 1)', '(3, 2)', '(3, 3)', '(3, 4)', '(3, 5)', '(3, 6)', '(3, 7)', '(3, 8)', '(3, 9)', '(3, 10)', '(3, 11)', '(3, 12)', '(3, 13)', '(3, 14)', '(3, 15)', '(3, 16)', '(3, 17)', '(3, 18)', '(3, 19)', '(4, 0)', '(4, 1)', '(4, 2)', '(4, 3)', '(4, 4)', '(4, 5)', '(4, 6)', '(4, 7)', '(4, 8)', '(4, 9)', '(4, 10)', '(4, 11)', '(4, 12)', '(4, 13)', '(4, 

In [ ]:
a = {**players[0].all_playable_moves}

In [ ]:
run = True
weights = [1,1,10]
turn = 0

player = players[p_i]


if False not in [player.isDone for player in players]: # when all players are done, game stops.
    run = False
    

if not player.isDone:
    matrix = bot.convert_matrix_to_nparray(board.matrix)
    possible_places, possible_plays_indices, possible_pieces = bot.get_available_actions(matrix, player)
    possible_places = [None if lst is None else tuple(lst) for lst in possible_places]
    if len(possible_plays_indices) == 0:
        player.isDone = True
    else:
        # Random play right now.
        best = 0
        choice = 0
        best_reward = None

        

        rand_index = bot.random_index(possible_plays_indices)
        pkey, pindex = possible_plays_indices[choice]
        absolute_coords = possible_places[choice]
        piece_played = possible_pieces[choice]
        #print(choice)
        bot.play_coordinates(pkey, pindex, absolute_coords, piece_played, player, board)
        if len(player.pieces) == 0:
            player.isDone = True
        print(bot.convert_matrix_to_nparray(board.matrix))

p_i+=1
if p_i>3: 
    p_i=0

In [ ]:
del corner_to

In [ ]:
def check_corner_on_pos(matrix,i,j):
    if i > 0 and j > 0 and i <19 and j <19:
        s = matrix[i-1:i+2,j-1:j+2]
        d = {s[0][0],s[0][2],s[2][0],s[2][2]}
        l = {s[1,0],s[1,2],s[0,1],s[2,1]}
        corner_to = d.difference(l).difference({0})
        return corner_to
    elif i == 0 and j >0 and j<19:
        s = matrix[i:i+2,j-1:j+2]
        d = {s[1][0],s[1][2]}
        l = {s[0,0],s[0,2],s[1,1]}
        corner_to = d.difference(l).difference({0})
    elif i == 19 and j > 0 and j < 19:
        s = matrix[i-1:i+1,j-1:j+2]
        d = {s[0][0],s[0][2]}
        l = {s[1,0],s[1,2],s[0,1]}
        corner_to = d.difference(l).difference({0})
    elif  j == 0 and i >0 and i<19:
        s = matrix[i-1:i+2,j:j+2]
        d = {s[0][1],s[2][1]}
        l = {s[0,0],s[1,1],s[2,0]}
        corner_to = d.difference(l).difference({0})
    elif  j == 19 and i >0 and i<19:
        s = matrix[i-1:i+2,j-1:j+1]
        d = {s[0][0],s[2][0]}
        l = {s[0,1],s[1,0],s[2,1]}
        corner_to = d.difference(l).difference({0})
    return corner_to

In [ ]:
set([0])

In [ ]:
a = np.array([[1, 0, 2],
       [0, 0, 1],
       [2, 0, 0]], dtype=np.int8)

In [ ]:
matrix

In [ ]:
s

In [ ]:
i,j = (1,1)
for n in range(1000000):
    s = matrix[i-1:i+2,j-1:j+2]
    d = {s[0][0],s[0][2],s[2][0],s[2][2]}
    l = {s[1,0],s[1,2],s[0,1],s[2,1]}
    corner_to = d.difference(l).difference({0})


In [ ]:
for n in range(1000000):
    np.where(a== 2)

In [ ]:
np.all(np.where(a.flatten() == 2)[0]% 2 == 0)

In [ ]:
a

In [ ]:
print(bot.convert_matrix_to_nparray(board.matrix))

In [ ]:

for n in range(10000):
    corners = {}
    for c in range(1,5):
        corners[c]= set(bot.get_playable_conditions(matrix,c, only_corners= True))

corners

In [ ]:
for n in range(10000):
    corners = {1:[],2:[],3:[],4:[]}
    for i,j in np.argwhere(matrix == 0):
        c = check_corner_on_pos(matrix,i,j)
        if len(c) > 0:
            for p in c:
                corners[p].append((i,j))

In [ ]:
a = np.array([[1, 0, 2],
       [0, 0, 1],
       [2, 0, 0]], dtype=np.int8)
b=a
c = np.array(a,copy=True)
a[0][0] = 5
c[2][2]= 5
print(a)
print(b)
print(c)

In [ ]:
for n in range(1000000):
    new_matrix = np.array(matrix, copy = True)

In [ ]:
p = possible_places[0:5]
print(p)
i = 2
p[:i] + p[i+1:]

In [ ]:
False not in [False,True]

In [ ]:
for i in range(3):
    for j in range(3):
        print(i,j)


In [ ]:
def evaluate_board(matrix, possible_places, color, p_color, corners):
    pass

def is_game_over(players):
    return False not in [player.isDone for player in players] #means all players have isDone True

def next_player(current_player):
    # Function to determine the next player in the sequence. current_player is an int between 1 and 4
    return (current_player + 1) % 4 +1 # will go from 1 to 4

In [ ]:
min(2,5)

In [ ]:
def get_last_n(lst, n):
    return lst[max(len(lst)-n,0):len(lst)]

In [ ]:
def check_corner_on_pos(matrix,i,j):
    if i > 0 and j > 0 and i <19 and j <19:
        s = matrix[i-1:i+2,j-1:j+2]
        d = {s[0][0],s[0][2],s[2][0],s[2][2]}
        l = {s[1,0],s[1,2],s[0,1],s[2,1]}
        corner_to = d.difference(l).difference({0})
    elif i == 0 and j >0 and j<19:
        s = matrix[i:i+2,j-1:j+2]
        d = {s[1][0],s[1][2]}
        l = {s[0,0],s[0,2],s[1,1]}
        corner_to = d.difference(l).difference({0})
    elif i == 19 and j > 0 and j < 19:
        s = matrix[i-1:i+1,j-1:j+2]
        d = {s[0][0],s[0][2]}
        l = {s[1,0],s[1,2],s[0,1]}
        corner_to = d.difference(l).difference({0})
    elif  j == 0 and i >0 and i<19:
        s = matrix[i-1:i+2,j:j+2]
        d = {s[0][1],s[2][1]}
        l = {s[0,0],s[1,1],s[2,0]}
        corner_to = d.difference(l).difference({0})
    elif  j == 19 and i >0 and i<19:
        s = matrix[i-1:i+2,j-1:j+1]
        d = {s[0][0],s[2][0]}
        l = {s[0,1],s[1,0],s[2,1]}
        corner_to = d.difference(l).difference({0})
    else:
        corner_to = set()
    return corner_to

def get_corners_for_all_players(matrix):
    corners = {1:[],2:[],3:[],4:[]}
    for i,j in np.argwhere(matrix == 0):
        c = check_corner_on_pos(matrix,i,j)
        if len(c) > 0:
            for p in c:
                corners[p].append((i,j))
    return corners

In [373]:
def choose_candidate_moves(matrix, possible_places, color, weights = [10,10,1], num_of_candidates = 5, max_places = 100):
    '''takes in 4 weights'''
    best = []
    choice = []
    corners = get_corners_for_all_players(matrix)
    shift = max(len(possible_places) - max_places,0)
    possible_places = possible_places[len(possible_places)-max_places:len(possible_places)]
    for i in range(len(possible_places)):
        reward = []
        reward.append(bot.get_number_of_blocked_corners(corners, possible_places[i], color))
        reward.append(bot.get_number_of_corners(matrix, possible_places[i], color, corners)) # negative value for own color. you want to add some.
        reward.append(len([coord for coord in possible_places[i] if coord is not None]))
        current_sum = sum([x * y for x, y in zip(weights, reward)])
        if len(best) < num_of_candidates:
            best.append(current_sum)
            choice.append(i + shift)
        elif current_sum > min(best):
            index_to_replace = best.index(min(best))
            best[index_to_replace] = current_sum
            choice[index_to_replace] = i + shift
    return choice

In [ ]:
a = [1,2,3,4,5,6,7]
a = a[len(a)-2:len(a)]
a

In [376]:
for n in range(100):
    possible_places = bot.get_available_actions(matrix, player)[0]
    choose_candidate_moves(matrix, possible_places, 3, weights = [10,10,1], num_of_candidates = 10, max_places=100)

In [365]:
def minimax_alpha_beta(state, depth, alpha, beta, current_player, max_player):
    if depth == 0 or is_game_over(state):
        return evaluate_board(state), None
    
    if current_player == max_player:
        possible_places = bot.get_available_actions(matrix, player)[0]
        
        max_eval = float('-inf')
        best_move = None
        for move in choose_candidate_moves(matrix,possible_places,current_player,num_of_candidates=10):
            new_state = bot.matrix_with_placed_piece(matrix,possible_places(move),current_player)  # Apply the move to get the new state
            eval, _ = minimax_alpha_beta(new_state, depth - 1, alpha, beta, next_player(current_player), max_player)
            if eval > max_eval:
                max_eval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break  # Beta pruning
        return max_eval, best_move
    else:
        min_eval = float('inf')
        best_move = None
        for move in bot.get_available_actions(state, current_player):
            new_state = bot.matrix_with_placed_piece(matrix,move,current_player)  # Apply the move to get the new state
            eval, _ = minimax_alpha_beta(new_state, depth - 1, alpha, beta, next_player(current_player), max_player)
            if eval < min_eval:
                min_eval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break  # Alpha pruning
        return min_eval, best_move
